## Preparing the arXiv ID for SS

In [2]:
import re 

datas = []
unmatch = []
# Regular expression pattern to find and capture the numerical parts
pattern = r"https://arxiv.org/pdf/(\d{4}\.\d{5})\.pdf"


with open('data/reason.md', 'r') as f:
    for line in f.readlines():  
        if 'https://arxiv' in line:
            # Find all matches in the string
            matches = re.findall(pattern, line)
            datas.append(f'ARXIV:{matches[0]}')
        elif 'pdf' in line:
            unmatch.append(line)

datas[:5]

['ARXIV:2309.13625',
 'ARXIV:2310.13023',
 'ARXIV:2310.04560',
 'ARXIV:2310.04944',
 'ARXIV:2310.00299']

## Get TL;DR from SemanticScholar

In [3]:
from semanticscholar import SemanticScholar
sch = SemanticScholar()

In [3]:
def get_meta_data(paper):
    domains = ['title', 'authors', 'abstract', 'tldr', 'venue', 'referenceCount', 'citationCount', 'influentialCitationCount']

    meta_data = {}
    for domain in domains:
        if domain in paper.keys():
            if domain == 'authors':
                authors = []
                for author_domain in paper.__getattribute__(domain):
                    authors.append(author_domain['name'])
                meta_data['authors'] = authors
            else:
                domain_res = paper.__getattribute__(domain)
                if domain == 'tldr':
                    domain_res = str(domain_res)
                meta_data[domain] = domain_res
    return meta_data

In [5]:
from tqdm import tqdm 
import time

meta_datas = []
with open('result.json', 'a+') as fp:
    for data in tqdm(datas):
        try:
            paper = sch.get_paper(data)
            fp.writelines(get_meta_data(paper))
        except:
            fp.writelines({'Cannot find': data})
        
        meta_datas.append(get_meta_data(paper))
        time.sleep(2) # hold 2 sec

  2%|▏         | 2/111 [00:22<22:41, 12.49s/it]

## arXiv searching

In [8]:
import arxiv

# Construct the default API client.
client = arxiv.Client()

# Search for the 10 most recent articles matching the keyword "quantum."
# -----
# sort_by
# Relevance 
# LastUpdatedDate 
# SubmittedDate
# -----
# order
# Ascending
# Descending

search = arxiv.Search(
  query = "Large Language Models",
  max_results = 10,
  sort_by = arxiv.SortCriterion.SubmittedDate,
  sort_order = arxiv.SortOrder.Descending,
)

results = client.results(search)
for r in client.results(search):
  print(r.title, r.published)

DistriFusion: Distributed Parallel Inference for High-Resolution Diffusion Models 2024-02-29 18:59:58+00:00
Single Electron Quantum Dot in Two-Dimensional Transition Metal Dichalcogenides 2024-02-29 18:59:57+00:00
Panda-70M: Captioning 70M Videos with Multiple Cross-Modality Teachers 2024-02-29 18:59:50+00:00
Learning a Generalized Physical Face Model From Data 2024-02-29 18:59:31+00:00
Impact of weak lensing on bright standard siren analyses 2024-02-29 18:59:30+00:00
The Counterfeit Conundrum: Can Code Language Models Grasp the Nuances of Their Incorrect Generations? 2024-02-29 18:59:25+00:00
The All-Seeing Project V2: Towards General Relation Comprehension of the Open World 2024-02-29 18:59:17+00:00
Retrieval-Augmented Generation for AI-Generated Content: A Survey 2024-02-29 18:59:01+00:00
Lifelong Benchmarks: Efficient Model Evaluation in an Era of Rapid Progress 2024-02-29 18:58:26+00:00
Loose LIPS Sink Ships: Asking Questions in Battleship with Language-Informed Program Sampling 2

In [2]:
import arxiv

# Construct the default API client.
client = arxiv.Client()

# Search for the 10 most recent articles matching the keyword "quantum."
search = arxiv.Search(
  query = "Large Language Models",
  max_results = 10,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

results = client.results(search)

# `results` is a generator; you can iterate over its elements one by one...
for r in client.results(search):
  print(r.title)
# ...or exhaust it into a list. Careful: this is slow for large results sets.
all_results = list(results)
print([r.title for r in all_results])

# For advanced query syntax documentation, see the arXiv API User Manual:
# https://arxiv.org/help/api/user-manual#query_details
search = arxiv.Search(query = "au:del_maestro AND ti:checkerboard")
first_result = next(client.results(search))
print(first_result)

# Search for the paper with ID "1605.08386v1"
search_by_id = arxiv.Search(id_list=["1605.08386v1"])
# Reuse client to fetch the paper, then print its title.
first_result = next(client.results(search))
print(first_result.title)

DistriFusion: Distributed Parallel Inference for High-Resolution Diffusion Models
Single Electron Quantum Dot in Two-Dimensional Transition Metal Dichalcogenides
Panda-70M: Captioning 70M Videos with Multiple Cross-Modality Teachers
Learning a Generalized Physical Face Model From Data
Impact of weak lensing on bright standard siren analyses
The Counterfeit Conundrum: Can Code Language Models Grasp the Nuances of Their Incorrect Generations?
The All-Seeing Project V2: Towards General Relation Comprehension of the Open World
Retrieval-Augmented Generation for AI-Generated Content: A Survey
Lifelong Benchmarks: Efficient Model Evaluation in an Era of Rapid Progress
Loose LIPS Sink Ships: Asking Questions in Battleship with Language-Informed Program Sampling
['DistriFusion: Distributed Parallel Inference for High-Resolution Diffusion Models', 'Single Electron Quantum Dot in Two-Dimensional Transition Metal Dichalcogenides', 'Panda-70M: Captioning 70M Videos with Multiple Cross-Modality Tea